<h1>Proyecto Tecnologías de Búsqueda en la Web</h1>
<h3>Integrantes</h3>
<ul><li>Sebastián Aranda</li><li>Felipe Santander</li></ul>

<h2>Librerías</h2>

In [2]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary, bleicorpus
from gensim.models import ldamodel
from gensim.models import Phrases

from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from random import randint

import pyLDAvis
import re
import os
import json

<h2>Creación del Corpus</h2>

In [3]:
corpus_path = "cyberbullying_corpus/"
roastme_dataset_path = "Corpus_builder_and_pre/Dataset/"
balance_dataset_path = "Corpus_builder_and_pre/Neutros&Positivos/"
#corpus_path = "corpus_lda/corpus_lda.lda_c"
#dictionary_path ="corpus_lda/corpus_lda.dict"

In [48]:
#Create bad words list
bad_words = []
with open('badwords','r') as bad_words_file:
    for word in bad_words_file:
        word = word.replace('\n','').decode('unicode_escape').encode('ascii','ignore')
        if word != '':
            bad_words.append(word)
print("Badwords List Size: "+str(len(bad_words)))
        
#Remove keywords in cyberbullying from stopwords list
keywords = ["you", "your", "he", "she", "it"]
stopword_list = [stopword for stopword in stopwords.words('english') if stopword not in keywords]
puncts = ".,:;?!()[]{}~+-\"\'#$%&\/"
digits = "0123456789"

#Create the corpus
print("Creating the corpus...")
mcorpus = []

print("Adding Roastme Dataset...")
max_num_docs = 5000
count = 0
for name in os.listdir(roastme_dataset_path):
    if count >= max_num_docs:
        break
    if name.endswith('.json'):
        with open(roastme_dataset_path+'/'+name) as f:
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:    
                    #Extrae el comentario
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    #Elimina los links
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)

                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")

                    tokens_comment = [word for word in comment_text.lower().split() if word not in stopword_list]
                    mcorpus.append(tokens_comment)
                    
                    count += 1
                    if count >= max_num_docs:
                        break
                        
                    try:
                        for child in child['data']['replies']['data']['children']:
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")
                    
                            tokens_comment = [word for word in comment_text.lower().split() if word not in stopword_list]
                            mcorpus.append(tokens_comment)
                            
                            count += 1
                            if count >= max_num_docs:
                                break
                                
                    except Exception:
                        pass
            except Exception:
                #pint('Exception at file:'+str(count)+' has no children')
                pass
print("RoastMe #Docs: "+str(count))

print("Adding Neutral and Positive Comments Dataset...")
max_num_docs = 5000
count = 0
for name in os.listdir(balance_dataset_path):
    if count >= max_num_docs:
        break
    if name.endswith('.json'):
        with open(balance_dataset_path+'/'+name) as f:
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:    
                    #Extrae el comentario
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    #Elimina los links
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)

                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")

                    tokens_comment = [word for word in comment_text.lower().split() if word not in stopwords.words('english')]
                    mcorpus.append(tokens_comment)
                    
                    count += 1
                    if count >= max_num_docs:
                        break
                    
                    try:
                        for child in child['data']['replies']['data']['children']:
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")
                    
                            tokens_comment = [word for word in comment_text.lower().split() if word not in stopwords.words('english')]
                            mcorpus.append(tokens_comment)
                            
                            count += 1
                            if count >= max_num_docs:
                                break
                    except Exception:
                        pass
            except Exception:
                #pint('Exception at file:'+str(count)+' has no children')
                pass
print("Neutral&Positive #Docs: "+str(count))

#Remove words that appear only once
frequency = defaultdict(int)
for doc in mcorpus:
    for token in doc:
        frequency[token] += 1
docs = [[token for token in doc if frequency[token] > 1] for doc in mcorpus]

# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            docs[idx].append(token)

#Create and save dictionary
dictionary = corpora.Dictionary(docs)
#dictionary.filter_extremes(no_below=20, no_above=0.5)

#Convert documents to vectors
corpus = [dictionary.doc2bow(doc) for doc in docs]

#Increasing weight of bad words:
bad_words_ids = [word_id for word_id, word in dictionary.iteritems() if word in bad_words]
ngrams_ids = [word_id for word_id, word in dictionary.iteritems() if "_" in word]

for doc_idx in range(len(corpus)):
    word_id_list = []
    freq_list = []

    for word_id, freq in corpus[doc_idx]:
        word_id_list.append(word_id)
        #if (word_id in bad_words_ids and randint(0,9) > 5):
        if (word_id in bad_words_ids):
            freq_list.append(freq*3)
        else:
            freq_list.append(freq)
    
    new_doc = zip(word_id_list,freq_list)
    corpus[doc_idx] = new_doc

print("Saving Corpus and Dictionary...")
dictionary.save('tmp/cyberbullying_dictionary.dict') #Save the dictionary
corpora.BleiCorpus.serialize('tmp/cyberbullying_corpus.lda-c', corpus) #Save the corpus

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))


Badwords List Size: 376
Creating the corpus...
Adding Roastme Dataset...
RoastMe #Docs: 5000
Adding Neutral and Positive Comments Dataset...
Neutral&Positive #Docs: 4629
Saving Corpus and Dictionary...
Number of unique tokens: 4872
Number of documents: 9629


In [47]:
#Load the dictionary and corpus
if (os.path.exists('tmp/cyberbullying_dictionary.dict') and os.path.exists('tmp/cyberbullying_corpus.lda-c')):
    print('Creating dictionary...')
    %time dictionary = corpora.Dictionary.load('tmp/cyberbullying_dictionary.dict')
    print('Creating corpus...')
    %time corpus = corpora.BleiCorpus('tmp/cyberbullying_corpus.lda-c')
else:
    print("Create the dictionary and corpus first")

Creating dictionary...
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 12.7 ms
Creating corpus...
CPU times: user 120 ms, sys: 4 ms, total: 124 ms
Wall time: 118 ms


In [46]:
#Initialize a model
#print('Creating Tfidf model...')
#%time tfidf = models.TfidfModel(corpus)
#corpus_tfidf = tfidf[corpus]

#Create a LDA Model and save it
%time lda_2 = ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary)
#%time lda_3 = ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary)
#%time lda_4 = ldamodel.LdaModel(corpus, num_topics=4, id2word=dictionary)
#%time lda_5 = ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary)
lda_2.save('tmp/cyberbullying_ldaModel.lda')

CPU times: user 33 s, sys: 16 ms, total: 33 s
Wall time: 33 s


In [ ]:
#Load LDA Model
lda = models.LdaModel.load('tmp/cyberbullying_ldaModel.lda')

In [47]:
#Print Word Topic Distribution for 100 words
lda_2.print_topics(2,100)
#lda_3.print_topics(3,100)

[(0,
  u'0.020*"you" + 0.012*"your" + 0.010*"friend" + 0.010*"make" + 0.008*"people" + 0.008*"love" + 0.007*"really" + 0.007*"r" + 0.007*"great" + 0.007*"know" + 0.007*"get" + 0.007*"like" + 0.006*"thank" + 0.006*"shit" + 0.006*"want" + 0.006*"think" + 0.006*"sub" + 0.005*"day" + 0.005*"life" + 0.005*"feel" + 0.005*"see" + 0.005*"say" + 0.005*"u" + 0.005*"thing" + 0.005*"hope" + 0.004*"good" + 0.004*"could" + 0.004*"go" + 0.004*"still" + 0.004*"well" + 0.004*"also" + 0.004*"always" + 0.004*"nice" + 0.004*"happy" + 0.004*"much" + 0.004*"fuck" + 0.004*"time" + 0.004*"better" + 0.004*"fucking" + 0.004*"post" + 0.004*"best" + 0.003*"someone" + 0.003*"even" + 0.003*"something" + 0.003*"look" + 0.003*"need" + 0.003*"lot" + 0.003*"work" + 0.003*"comic" + 0.003*"take" + 0.003*"give" + 0.003*"way" + 0.003*"actually" + 0.003*"got" + 0.003*"one" + 0.003*"dad" + 0.003*"bad" + 0.003*"though" + 0.003*"help" + 0.003*"find" + 0.003*"wholesome" + 0.003*"mean" + 0.003*"would" + 0.003*"never" + 0.002*"wo

In [38]:
#Get top 10 terms of topic 0
for word_id, prob in lda.get_topic_terms(1,10):
    print(dictionary[word_id])

you
year_old
your
fuck
it
love_child
first_time
he
one
get


In [44]:
texts = ["you look like an asshole sitting there in the middle of the room",
        "you are a fucking slut",
         "your penis makes me laugh",
         "you look like shit",
         "thanks for you help",
         "you look great",
        "that is a great work"]


tcorpus = [dictionary.doc2bow(doc.lower().split()) for doc in texts]
print("\t Agressive \t\t Non-Agressive")
for docBow in tcorpus:
    print(lda_2[docBow]) 
print("-----------")

	 Agressive 		 Non-Agressive
[(0, 0.08573602194025054), (1, 0.91426397805974935)]
[(0, 0.4497260134803262), (1, 0.55027398651967385)]
[(0, 0.80782673150485518), (1, 0.19217326849514479)]
[(0, 0.48843112865252192), (1, 0.51156887134747808)]
[(0, 0.85106364180749061), (1, 0.14893635819250942)]
[(0, 0.61612410328736078), (1, 0.38387589671263922)]
[(0, 0.8004721583407165), (1, 0.19952784165928347)]
-----------
